# Kiltisbotin kulutusdata

Kiltisbot on Fyysikkokillan kiltistoimikunnan ylläpitämä Telegramin bottirajapintaan pohjautuva multimedia ratkaisu, jonka tarkoitus on helpottaa kiltalaisten elämää. Kiltibotissa on tällä hetkellä integraatiot niin Fiirumille kuin Fyysikkokillan Googlekalenteriinkin. Kiltisbotin alkuperäinen ja tärkein käyttötarkoitus on kiltiksen herkkukaapin piikkimyynnin siirtäminen 2000-luvulle eli paperisesta piikkilistasta pilveen.

Kiltisbot otettiin käyttöön 13.4.2019 ja tätä juttua varten on käytetty dataa yhden vuoden ajalta 13.4.2019 - 12.4.2020. 

In [ ]:
# Assumes that your ssh key is in the guild's server. Contact tech support if needed.
!scp root@104.248.244.199:/root/fk-kiltisbot/kiltis_PROD.db .

In [ ]:
import datetime

import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statistics as stats

from scipy.stats import norm, normaltest

In [ ]:
conn = sqlite3.connect("kiltis_PROD.db")

In [ ]:
sql = """
SELECT *
FROM transactions
"""
data = pd.read_sql(sql, conn)

In [ ]:
data["datetime"] = data.aika.astype("datetime64")
data["date"] = data.aika.apply(lambda x: x[:10]).astype("datetime64")
data["weekday"] = data.date.apply(lambda x: x.isocalendar()[2])
data["week"] = data.date.apply(lambda x: x.isocalendar()[1])
data["minute"] = data.aika.apply(lambda x: int(x.split(":")[1]))
data["hour"] = data.datetime.apply(lambda x: x.hour)

In [ ]:
nostot = data[data.tuote == "NOSTO"]
panot = data[data.tuote == "PANO"]
ostot = data[(data.tuote != "NOSTO") & (data.tuote != "PANO")]

In [ ]:
print("Tapahtumia yhteensä: {}".format(len(data)))
print("Panoja {} kappaletta.  Yhteensä {} euroa.".format(len(panot), panot.hinta.sum() / 100))
print("Nostoja {} kappaletta.   Yhteensä {} euroa.".format(len(nostot), nostot.hinta.sum() / 100))
print("Ostoja {} kappaletta. Yhteensä {:.2f} euroa.".format(len(ostot), ostot.hinta.sum() / 100))
print("Sähköisen kiltispiikin tase: {} euroa".format((panot.hinta.sum() - nostot.hinta.sum() - ostot.hinta.sum()) / 100))

## Tuotteiden kokonaiskysyntä

In [ ]:
tuotteittainen = ostot.groupby("tuote").count().sort_values("id", ascending = True).reset_index()
plt.figure(1, (5, 14))
plt.barh(tuotteittainen.tuote, tuotteittainen.id)
plt.grid()
#plt.tight_layout()

## Päivittäiset kulutuksen vaihtelu vuoden sisällä

In [ ]:
paivat = pd.DataFrame(columns=["date"], data=pd.date_range(start="2019-04-13", end="2020-04-12"))
paivittainen = paivat.merge(ostot, how="left", on="date").groupby("date").count().reset_index()

In [ ]:
print("Keskiarvo: {:.2f}".format(paivittainen.tuote.mean()))
print("Keskihajonta: {:.2f}".format(paivittainen.tuote.std()))
print("Maksimi: {}".format(paivittainen.tuote.max()))
print("Minimi: {}".format(paivittainen.tuote.min()))

In [ ]:
plt.figure(1, (14, 8))
plt.plot(paivittainen.date, paivittainen.tuote)
plt.title("Kiltiksen päivittäinen kulutus vuoden aikana", size=16)
plt.ylabel("Ostettuja tuotteita", size=14)
#plt.xticks(15 + np.arange(12) * 30, kuukaudet)
plt.show()

In [ ]:
fig=plt.figure(1, (14, 5))
ax = fig.add_axes([0,0,1,1])
ax.plot(paivittainen.date, paivittainen.tuote.rolling(14).mean(),linewidth=4)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.tick_params(axis='both', labelsize=22)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)
plt.title("Kahden viikon liukuva keskiarvo", size=22)
plt.ylabel("Tuotetta", size=22)
plt.show()

Maksimi 32 ostosta päivässä kirjattiin päivälle, jolloin järjestettiin myös killan vuoden 2020 vuosikokous.

## Kulutuksen vaihtelu viikon sisällä 

In [ ]:
viikoittainen = ostot.groupby("weekday").count().reset_index()

In [ ]:
#plt.figure(1, (4, 6))
fig, ax = plt.subplots()
ax.stem(np.arange(7), viikoittainen.tuote / 52)
#plt.barh(np.arange(7), viikoittainen.tuote / 52)
ax.set_xticklabels(["", "ma", "ti", "ke", "to", "pe", "la", "su"])
#ax.set_xlim(8,0)
ax.tick_params(axis='both', labelsize=12)
#ax.invert_yaxis() 
#ax.invert_xaxis()
#ax.xaxis.tick_top()
#ax.yaxis.tick_right()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.tick_params(length = 0)
#ax.get_yaxis().yticks(np.arange(7), ["ma", "ti", "ke", "to", "pe", "la", "su"], size=17)
plt.title("Keskimääräinen kulutus viikonpäivinä", size=18)
#plt.ylabel("Tuotetta", size=14)
#plt.grid(color="white", axis="x")
plt.show()

## Viikoittaisen kulutuksen vaihtelu vuoden sisällä

In [ ]:
weeks = pd.DataFrame(columns=["week"], data = 1 + (np.arange(0, 52) + 14) % 52)
viikot = weeks.merge(ostot, how="left", on="week").groupby("week").count().reset_index()

In [ ]:
plt.figure(1, (14, 8))
plt.bar(viikot.week, viikot.tuote)
plt.title("Kiltiksen viikoittainen kulutus", size=16)
plt.ylabel("Tuotteita", size=16)
plt.xlabel("Viikko", size=16)
plt.xlim(0, 53)

## Minuutittainen tarkastelu

In [ ]:
minuutittaiset = ostot.groupby("minute").count().reset_index()

In [ ]:
plt.figure(1, (14, 8))
plt.bar(minuutittaiset.minute, minuutittaiset.tuote)
plt.xticks(5 + np.arange(12) * 5, size=14)
plt.ylabel("Tuotetta", size=14)
plt.xlabel("Minuutti", size=14)
plt.title("Myynti tunnin aikana", size=16)
plt.show()

In [ ]:
daily = ostot.hour * 60 + ostot.minute

In [ ]:
x = np.linspace(0, 1440, 1000)
mu, std = norm.fit(daily)
p = norm.pdf(x, mu, std)

In [ ]:
fig = plt.figure(1, (14, 8))
ax = fig.add_axes([0,0,1,1])

plt.hist(daily, density=True, bins=30, alpha=0.6)
plt.plot(x, p, linewidth=2, color="k")
plt.title("Nomaalijakauma sovitettuna päivittäiseen kulutusdataan", size=20)
plt.ylabel("Tiheys", size=20)
plt.xlabel("Minuutti", size=20)

ax.tick_params(axis='both', labelsize=20)
ax.tick_params(length = 0)
plt.axhline(0, color="red")
#plt.grid(axis='y')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)

In [ ]:
mu

In [ ]:
tuotteet = list(set(data["tuote"]))
kayttajat = list(set(data["user"]))
m = np.zeros((len(tuotteet), len(kayttajat)))

In [ ]:
def get_count(user, tuote):
    sql = """
    SELECT 
    COUNT(*)
    FROM transactions
    WHERE user = '{}'
    AND tuote = '{}'
    """.format(user, tuote)
    d = pd.read_sql(sql, conn)
    return d.iloc[0,0]

In [ ]:
for i, t in enumerate(tuotteet):
    for j, k in enumerate(kayttajat):
        m[i, j] = get_count(k, t)

In [ ]:
corr = np.corrcoef(m)

f, ax = plt.subplots(figsize=(11, 9))
cmap = sns.diverging_palette(220, 10, as_cmap=True)
sns.heatmap(corr, cmap=cmap, center=0, square=True, linewidths=.5, cbar_kws={"shrink": .5}, xticklabels=tuotteet, yticklabels=tuotteet)

## Kiltispiikin viikonloppukäyttö

Kiltalaiset jakautuvat kahteen kastiin: niihin joilla on avain kiltahuoneelle ja niihin joilta tämä saavutettu etuus puuttuu. Tutkittaessa kuinka piikin viikonloppukäyttö suhtautuu sen kokonaiskäyttöön, tämä joukko käy erottuu.

In [ ]:
#TODO legend kilta eliitti vs kilta plebeijit
vkl_ostot = ostot[(ostot["weekday"] == 7) | (ostot["weekday"] == 6)]
vkl = vkl_ostot["user"].value_counts()
tot = ostot["user"].value_counts()

x = tot.values
y = [vkl[uid] if uid in vkl.keys() else 0 for uid in tot.keys()]
colors = ['r' if point >= 10 else 'b' for point in y]

plt.figure(1, (12, 8))
plt.scatter(x,y,c=colors,marker='o')
plt.grid()
plt.xlabel('Kokonaiskäyttö')
plt.ylabel('Viikonloppukäyttö')
plt.show()

## Eri tuotteiden mediaaniajat

Mediaanikiltalainen popsii muumilakun klo 11, juo Power Kingin klo 14, hörpää VitaminWaterin naamariin klo 15 ja skippaa päivällisen ja syö kiltahuoneella nuudelit ja fitness patukan hieman klo 17 jälkeen -- tämä kaikki tapahtuu tietenkin herkeämättömän kahvin ja teen ryystämisen ohella. Ohessa on esitetty mediaaniajat, jolloin osto on kirjattu bottiin.

In [ ]:
# 17.1.2020 server was changed which causes an offset of 2 hours
aikakorjatut_ostot = ostot.copy()
aikakorjatut_ostot.loc[ostot["date"] > np.datetime64('2020-01-17'), "hour"] += 2
aikakorjatut_ostot.loc[ostot["date"] > np.datetime64('2020-01-17'), "hour"] %= 24
aikakorjatut_ostot["quarter"] = 4 * aikakorjatut_ostot["hour"] + aikakorjatut_ostot["minute"] // 15

res = {}
products = ostot["tuote"].unique()
for p in products:
    d = aikakorjatut_ostot[aikakorjatut_ostot["tuote"] == p]
    res[p] = stats.median(d["quarter"])
res = {k: v for k, v in sorted(res.items(), key=lambda item: item[1])} # Sort by value

x = list(res.values())
y = list(res.keys())

x = x[2:]
y = y[2:]

# Plot timeline
levels = np.array([-9,9,-7,7,-5, 5, -3, 3, -1, 1])
fig, ax = plt.subplots(figsize=(25, 12))

# Create the base line
start = min(x)
stop = max(x)
ax.plot((x, x), (0, 0), 'k', alpha=.5)

# Iterate through median values annotating each one
for ii, (yy, xx) in enumerate(zip(y, x)):
    level = levels[ii % 10]
    vert = 'top' if level < 0 else 'bottom'

    ax.scatter(xx, 0, s=100, facecolor='w', edgecolor='k', zorder=9999)
    # Plot a line up to the text
    ax.plot((xx, xx), (0, level), c='r', alpha=.7)
    # Give the text a faint background and align it properly
    ax.text(xx, level, yy,
            horizontalalignment='right', verticalalignment=vert, fontsize=24,
            backgroundcolor=(1., 1., 1., .3))
#ax.set(title="Ostojen mediaaniajat")

# Remove components for a cleaner look
plt.setp((ax.get_yticklabels() + ax.get_yticklines() +
          list(ax.spines.values())), visible=False)

xtick_labels = [int(y // 4) for y in x]
xtick_pos = [4*y for y in xtick_labels]
plt.xticks(ticks=xtick_pos, labels=xtick_labels, fontsize=25,)
plt.show()

## Kiltispiikin käytön jakautuminen

Pieni joukko kiltalaisia käyttää bottia erityisen ahkerasti.

In [ ]:
res = ostot["user"].value_counts()
x = range(len(res))

plt.figure(1, (14, 8))
plt.bar(x,res.values)
plt.ylabel("Rappion määrä");
plt.xlabel("Anonymisoitu kiltalainen");
plt.tick_params(
    axis='x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False) # labels along the bottom edge are off
plt.grid()
#plt.axis('off') # remove all labels, add later with other software 
plt.show()

## Kiltahuoneen huoltovarmuuden turvaaminen

Ei tarvitse olla Supply Chain Analyst RELEXillä nähdäkseen seuraavista kuvaajista, että jos tavara on loppu, vaikuttaa se negatiivisesti tuotteen myyntiin.

In [ ]:
#tuotteet = ["Pätkis ", "Daim", "Twix", "Tupla Max", "Fitness-patukka"]
#ymax = 5

tuotteet = np.unique(ostot.tuote.values)
ymax = 12

fig, axs = plt.subplots(len(tuotteet), figsize=(15,len(tuotteet)))
paivat = pd.DataFrame(columns=["date"], data=pd.date_range(start="2019-04-13", end="2020-04-12"))

for i, tuote in enumerate(tuotteet):
    df = ostot[ostot.tuote == tuote]
    paivittainen = paivat.merge(df, how="left", on="date").groupby("date").count().reset_index()

    axs[i].plot(paivittainen.date, paivittainen.tuote, linewidth=4, color="white")
    axs[i].spines['top'].set_visible(False)
    axs[i].spines['right'].set_visible(False)
    axs[i].spines['bottom'].set_visible(False)
    axs[i].spines['left'].set_visible(False)
    axs[i].get_xaxis().set_ticks([])
    axs[i].get_yaxis().set_ticks([])
    axs[i].set_ylim((0,ymax))
    axs[i].set_ylabel(tuote, rotation=0, size=25,  horizontalalignment='right', )
    axs[i].yaxis.set_label_coords(-0.03,0)

plt.gcf().subplots_adjust(left=-0.15)
plt.show()
fig.savefig('temp.png', transparent=True, dpi=300, pad_inches=1)

## All in one master plot

Seuraava heat map antaa karkean kuvan mitä tuotteita menee erityisesti. Mitä enemmän punaista, sitä suurempi kulutus. Tulkinnassa kannattaa muistaa että osa tuotteista on saattanut olla vain lyhyen aikaa saatavilla. Marginaalijakaumat näyttävät käyttäjien kokonaiskulutuksen ja tuotteiden kulutuksen jakaumat.

In [ ]:
users = ostot.user.value_counts().keys()
tuotteet = ostot.tuote.value_counts().keys()

matrix = np.zeros(shape=(len(tuotteet), len(users)))

for i, tuote in enumerate(tuotteet):
    for j, user in enumerate(users):
        matrix[i,j] = len(ostot[(ostot.tuote == tuote) & (ostot.user == user)])

In [ ]:
from matplotlib import colors
from matplotlib.ticker import NullFormatter
nullfmt = NullFormatter() 
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

In [ ]:
plt.figure(1, figsize=(10, 10))

left, width = 0.1, 0.65
bottom, height = 0.1, 0.65
bottom_h = left_h = left + width + 0.02

rect_scatter = [left, bottom, width, height]
rect_histx = [left, bottom_h, width, 0.5]
rect_histy = [left_h, bottom, 0.5, height]

axHeatmap = plt.axes(rect_scatter)
axBarvert = plt.axes(rect_histx, frameon=False)
axBarhor = plt.axes(rect_histy, frameon=False)

axBarvert.xaxis.set_major_formatter(nullfmt)
axBarhor.yaxis.set_major_formatter(nullfmt)

# First plot heatmap of all sales
colmap = colors.ListedColormap(["white", "#ffeeee", "#ffdddd", "#ffbbbb", "#ff7777","#ff0000"])
bounds=[0,1,5,10,20,40,80]
norm = colors.BoundaryNorm(bounds, 6)
im = axHeatmap.pcolor(matrix, norm=norm, cmap=colmap)
axins = inset_axes(axHeatmap,
                   width="1%",  
                   height="50%", 
                   loc='lower left',
                   bbox_to_anchor=(-0.24, 0., 1, 1),
                   bbox_transform=ax.transAxes,
                   borderpad=0,
                   )
fig.colorbar(im, cax=axins)

axHeatmap.set_xticks(np.arange(start=0.5, stop=len(users), step=10))
axHeatmap.set_yticks(np.arange(start=0.5, stop=len(tuotteet)))

axHeatmap.set_xticklabels(np.arange(start=0, stop=len(users), step=10))
axHeatmap.set_yticklabels(tuotteet)

axHeatmap.set_xlabel("Anonymisoitu käyttäjä")

users_total = ostot.user.value_counts()
tuotteet_total = ostot.tuote.value_counts()

x1 = range(len(users_total))
x2 = range(len(tuotteet_total))

x1 = [x+0.5 for x in x1] # Correction so all bars show completely
x2 = [x+0.5 for x in x2]

# Bar charts for total consumtion per user and per product
axBarvert.bar(x1,users_total.values)
axBarhor.barh(x2,tuotteet_total.values)

axBarhor.tick_params(length = 0)
axBarvert.tick_params(length = 0)

axBarvert.grid(color="white")
axBarhor.grid(color="white")

axBarvert.set_xlim(axHeatmap.get_xlim())
axBarhor.set_ylim(axHeatmap.get_ylim())

plt.show()

## Kiltalaisten piikin kehitys

In [ ]:
def kayttajan_kulutus(kayttaja):
    kayttajan_data = data[data.user == kayttaja].copy()
    hinta_tuote = zip(kayttajan_data.tuote, kayttajan_data.hinta / 100)
    kayttajan_data["oikea_hinta"] = list(map(lambda x: x[1] if x[0] == "PANO" else -x[1], hinta_tuote))
    paivat = pd.date_range(start="2019-04-12", end="2020-03-20").to_series()
    kulutukset = paivat.apply(lambda x: kayttajan_data[kayttajan_data.date < x].oikea_hinta.sum())
    return (paivat, kulutukset)

# list of telegram user ids
#jabat =  

texts = ["Ugly", "Good", "Bad"]
saldot = [kayttajan_kulutus(jaba) for jaba in jabat]

fig = plt.figure(1, (14, 8))
ax = fig.add_axes([0,0,1,1])

for i, jaba in enumerate(saldot):
    x,y = jaba[0], jaba[1]
    plt.plot(x, y, linewidth=3)
    plt.text(x[-1], y[-1], " "+texts[i], size=22)

plt.title("The Good, the Bad and the Ugly", size=22)
plt.ylabel('Saldo euroissa', fontsize=22)
ax.tick_params(axis='both', labelsize=22)
ax.tick_params(length = 0)
plt.axhline(0, color="red")
#plt.grid(axis='y')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)